In [2]:
# Test the dataset

from lircst_ana_dataset import LircstAnaDataset 

def test_dataset():
    dataset = LircstAnaDataset('/home/samnub/dev/lircst-ana/data/')
    print(len(dataset))
    phan, sino, phantom_id = dataset[0]
    print(phan.shape, sino.shape, phantom_id)

test_dataset()

1797
(2, 128, 128) (128, 200, 100) 1742413505


In [1]:
# Fine-tune ControlNet using pytorch-lightning on our dataset

import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '/home/samnub/dev/lircst-diffusion/src/ControlNet')

from ControlNet.cldm.model import create_model, load_state_dict
from ControlNet.cldm.logger import ImageLogger
from torch.utils.data import DataLoader
from torchvision import transforms
import pytorch_lightning as pl

from ControlNet.tutorial_dataset import MyDataset
from lircst_ana_dataset import LircstAnaDataset

# Configs
resume_path = '/home/samnub/dev/lircst-diffusion/models/control_sd21_ini.ckpt'
batch_size = 4
logger_freq = 300
learning_rate = 1e-5
sd_locked = True
only_mid_control = False


# First use cpu to load models. Pytorch Lightning will automatically move it to GPUs.
model = create_model('/home/samnub/dev/lircst-diffusion/models/cldm_v21.yaml').cpu()
model.load_state_dict(load_state_dict(resume_path, location='cpu'))
model.learning_rate = learning_rate
model.sd_locked = sd_locked
model.only_mid_control = only_mid_control


# Misc
transform_phan = transforms.Compose([transforms.ToTensor(),
                                     ])
transform_sino = transforms.Compose([transforms.ToTensor(),
                                     ])
# dataset = LircstAnaDataset(data_dir='/home/samnub/dev/lircst-ana/data/')
dataset = MyDataset()
dataloader = DataLoader(dataset, num_workers=0, batch_size=batch_size, shuffle=True)
logger = ImageLogger(batch_frequency=logger_freq)
trainer = pl.Trainer(gpus=1, precision=32, callbacks=[logger])


# Train!
trainer.fit(model, dataloader)

/home/samnub/anaconda3/envs/lircst-diffusion-controlnet/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
/home/samnub/anaconda3/envs/lircst-diffusion-controlnet/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/samnub/anaconda3/envs/lircst-diffusion-controlnet/lib/python3.8/site-packages/pytorch_lightning/plugins/training_type/ddp.py:68: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import DistributedOptimizer


No module 'xformers'. Proceeding without it.
ControlLDM: Running in eps-prediction mode


/home/samnub/anaconda3/envs/lircst-diffusion-controlnet/lib/python3.8/site-packages/pytorch_lightning/core/lightning.py:2058: DeprecationWarning: `torch.distributed._sharded_tensor` will be deprecated, use `torch.distributed._shard.sharded_tensor` instead
  from torch.distributed._sharded_tensor import pre_load_state_dict_hook, state_dict_hook


DiffusionWrapper has 865.91 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


/home/samnub/anaconda3/envs/lircst-diffusion-controlnet/lib/python3.8/site-packages/open_clip/factory.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch

...

/home/samnub/anaconda3/envs/lircst-diffusion-controlnet/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:56: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 4. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/home/samnub/dev/lircst-diffusion/src/ControlNet/ldm/modules/diffusionmodules/util.py:136: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  torch.cuda.amp.autocast(**ctx.gpu_autocast_kwargs):


Data shape for DDIM sampling is (4, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [19:27<00:00, 23.34s/it]


Epoch 0:   0%|          | 28/12500 [42:41<316:56:52, 91.49s/it, loss=0.00705, v_num=1, train/loss_simple_step=0.00839, train/loss_vlb_step=4.68e-5, train/loss_step=0.00839, global_step=27.00]  

/home/samnub/anaconda3/envs/lircst-diffusion-controlnet/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:685: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
